<div style="text-align: right; font-size: 0.8em; color: #666;">
最終更新: 2026-02-17 19:30 (Speed & DynamicCache fix)
</div>

# 05. LLMベースの軽量マルチエージェント（Executor/Critic）

このノートブックでは、LLM（3Bクラス）を使用して、**「実行者（Executor）」**と**「批評者（Critic）」**の2つの役割を連携させる最小構成のエージェントシステムを体験します。

### なぜ2ステップなのか？
小型のLLM（3Bクラス）では、役割が多すぎるとコンテキスト（記憶）が混乱しやすくなります。「計画・実行」を Executor が一気に行い、それを Critic が「客観的に検証する」という2ステップに絞ることで、安定した自己反省（Self-Reflection）プロセスを実現します。


In [ ]:
# 編集禁止セル
import os
import sys
import torch

repo_path = '/content/llm_lab'
# キャッシュ回避用タグ
UPDATE_TAG = '20260217_1930'

if os.path.exists(repo_path):
    !rm -rf {repo_path}

!git clone -b ai_agent https://github.com/akio-kobayashi/llm_lab.git {repo_path}
%cd {repo_path}

!pip install -q -U transformers accelerate bitsandbytes

if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.common import load_llm
from src.agent_core import LLMExecutorCriticAgent

print(f'セットアップ完了 (Tag: {UPDATE_TAG})')


In [ ]:
# 編集禁止セル
model, tokenizer = load_llm(use_4bit=False)

# --- DynamicCache対策のパッチ適用 ---
from transformers.cache_utils import DynamicCache

orig_forward = model.forward
def patched_forward(*args, **kwargs):
    """DynamicCacheを古いタプル形式に変換してモデルに渡すパッチ"""
    if "past_key_values" in kwargs and isinstance(kwargs["past_key_values"], DynamicCache):
        kwargs["past_key_values"] = kwargs["past_key_values"].to_legacy_cache()
    return orig_forward(*args, **kwargs)

model.forward = patched_forward
if hasattr(model, "config"):
    model.config.use_cache = True # キャッシュを有効化して高速化
print("DynamicCache compatibility patch applied. Cache enabled.")
# ----------------------------------

def llm_chat(system_prompt: str, user_prompt: str, max_new_tokens: int = 512, temperature: float = 0.2):
    """安定・高速動作用チャット関数。"""
    try:
        prompt = (
            "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n"
            f"### システム:\n{system_prompt}\n\n"
            f"### 指示:\n{user_prompt}\n\n"
            "### 応答:\n"
        )

        inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=2048, truncation_side='left')
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        prompt_len = inputs['input_ids'].shape[1]

        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        # 1st pass: greedy (高速なキャッシュ生成を使用)
        with torch.inference_mode():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                min_new_tokens=1,
                do_sample=False,
                use_cache=True,
                pad_token_id=pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.05,
            )

        new_ids = gen_ids[0][prompt_len:]
        text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        # 2nd pass: fallback for empty
        if not text:
            with torch.inference_mode():
                gen_ids = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    min_new_tokens=16,
                    do_sample=True,
                    temperature=max(temperature, 0.7),
                    top_p=0.9,
                    use_cache=True,
                    pad_token_id=pad_token_id,
                    eos_token_id=None,
                    repetition_penalty=1.1,
                )
            new_ids = gen_ids[0][prompt_len:]
            text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        return text if text else '[Empty output]'
    except Exception as e:
        return f'Error: {e}'

agent = LLMExecutorCriticAgent(llm_chat)

queries = [
    'Pythonで数値を2倍する関数の最小コードを書いて。',
    '2026年2月の最新アニメ情報を要約して。情報が足りない場合は前提を明示して。',
]

for q in queries:
    final_answer, full_log, steps = agent.run_pipeline(q)
    print('=' * 70)
    print('Q:', q)
    print('--- Full Pipeline Log ---')
    print(full_log)
    print('=' * 70)


## 演習

1. **プロンプトエンジニアリング**: `src/agent_core.py` 内の `Critic` の `system_prompt` を変更して、より厳しく、あるいはより優しく指摘するように調整してみましょう。
2. **温度（Temperature）の調整**: `RoleConfig` の `temperature` を変更して、Executor の回答の「創造性」と Critic の「正確性」がどう変化するか観察してください。
